In [ ]:
#Code to mount Google Drive at Colab Notebook instance
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# Huggingface libraries to run LLM.
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

#LangChain related libraries
!pip install -q -U langchain

#Open-source pure-python PDF library capable of splitting, merging, cropping,
#and transforming the pages of PDF files
!pip install -q -U pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.6/229.6 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.9/283.9 kB 5.4 MB/s eta 0:00:00


In [ ]:
#from typing import List
import os
import time
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig

import torch

from langchain.llms import HuggingFacePipeline


device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))


Device: cuda
Tesla T4


In [ ]:
origin_model_path = "mistralai/Mistral-7B-Instruct-v0.1"
model_path = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"
bnb_config = BitsAndBytesConfig \
              (
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16,
              )
model = AutoModelForCausalLM.from_pretrained (model_path, trust_remote_code=True,
                                              quantization_config=bnb_config,
                                              device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(origin_model_path)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00008.bin:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

pytorch_model-00002-of-00008.bin:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

pytorch_model-00003-of-00008.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:

text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    batch_size=4,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=600,
    temperature = 0.3,
    do_sample=True,
)
text_generation_pipeline.tokenizer.pad_token_id = text_generation_pipeline.model.config.eos_token_id
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [ ]:
!pip install -q -U tiktoken
!pip install -q -U openai

**OPTION 1**

TEXT FITS IN CONTEXT WINDOW


In [ ]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [ ]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema.document import Document


def stuff_text_summarization():
  text = """It was a Thursday, but it felt like a Monday to John. And John loved Mondays. He thrived at work. He dismissed the old cliché of dreading Monday mornings and refused to engage in water-cooler complaints about “the grind” and empty conversations that included the familiar parry “How was your weekend?” “Too short!”. Yes, John liked his work and was unashamed.

  I should probably get another latte. I’ve just been sitting here with this empty cup. But then I’ll start to get jittery. I’ll get a decaf. No, that’s stupid, it feels stupid to pay for a decaf. I can’t justify that.

  John was always impatient on the weekends; he missed the formal structure of the business week. When he was younger he used to stay late after school on Fridays and come in early on Mondays, a pattern his mother referred to with equal parts admiration and disdain as “studying overtime.”

  Jesus, I’ve written another loser.

  Now, John spent his weekends doing yard work at the Tudor house Rebecca left him after their divorce. Rebecca, with her almond eyes—both in shape and in color—could never be his enemy.

  That barista keeps looking at me. She’ll probably ask me to leave if I don’t buy something. She’s kind of attractive. Not her hair—her hair seems stringy—but her face is nice. I should really buy something.

  Their divorce was remarkably amicable. In fact, John would often tell his parents, “Rebecca and I are better friends now than when we were married!” In fact, John looked forward to the days when he and Rebecca, with their new partners, would reminisce about their marriage, seeing it in a positive light, like two mature adults.

  Maybe I’ll just get a pumpkin-spice loaf. That way I can still sit here without going through a whole production of buying a coffee and giving my name and feeling like an asshole while it gets made.

  But if John were being honest, the house did get lonely on the weekends. Rebecca’s parents had been generous enough to leave John the house even though they had paid for it. John was still struggling to get his short-story writing—I mean, his painting—career off the ground, and Rebecca and her family had been more than supportive, even during the breakup."""

  token_count = num_tokens_from_string(text, "cl100k_base")
  print(f'Token Count: {token_count}')

  #Converting text to LangChain documents so that StuffDocumentsChain can understand Input
  documents = Document(page_content=text, metadata={"source": "local"})

  # Define prompt with prompt template
  prompt_template = """Write a concise summary of the following:
  "{docs}"
  CONCISE SUMMARY:"""
  prompt = PromptTemplate.from_template(prompt_template)

  # Define LLM chain
  llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

  # Define StuffDocumentsChain
  stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="docs")

  #Get the Summary of text by invoking StuffDocumentsChain
  summary = stuff_chain.invoke([documents])
  return summary['output_text']

In [ ]:
stuff_text_summarization()

Token Count: 487


" John enjoys his job and looks forward to Monday mornings. He misses the structure of the weekdays on weekends and does yard work at his ex-wife's house. Despite their divorce, John and Rebecca remain friendly and look forward to reminiscing about their marriage in the future."

OPTION 2

MAP REDUCE

In [ ]:
from langchain.document_loaders import PyPDFLoader

def load_pdf():
    # Load the pdf file
    loader = PyPDFLoader('/content/drive/MyDrive/ImpactofIndianPremierLeagueonTestCricketinIndia.pdf')
    documents = loader.load()
    token_count = num_tokens_from_string(str(documents), "cl100k_base")
    print(f'PDF Token Count: {token_count}')
    return documents, token_count

In [ ]:

from langchain.chains import MapReduceDocumentsChain, LLMChain, ReduceDocumentsChain, StuffDocumentsChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate

def map_reduce_summarize_document():

    #Load PDF using PyPDF
    documents, token_count = load_pdf()

    # Map Prompt template and LLM Chain
    map_prompt_template = """[INST] The following is a part of an pdf document:
    {docs}
    Based on this, please identify the main points.
    Answer:  [/INST] """
    map_prompt = PromptTemplate.from_template(map_prompt_template)
    map_chain = LLMChain(llm=mistral_llm, prompt=map_prompt)


    # Reduce Prompt template and LLM Chain
    reduce_prompt_template = """[INST] The following is set of summaries from the article:
    {doc_summaries}
    Take these and distill it into a final, consolidated summary of the main points.
    Construct it as a well organized summary of the main points and should be between 3 and 5 paragraphs.
    Answer:  [/INST] """
    reduce_prompt = PromptTemplate.from_template(reduce_prompt_template)
    reduce_chain = LLMChain(llm=mistral_llm, prompt=reduce_prompt)

    # Takes a list of documents, combines them into a single string, and passes this to an LLMChain
    combine_documents_chain = StuffDocumentsChain(
        llm_chain=reduce_chain, document_variable_name="doc_summaries"
    )

    # Combines and iteratively reduces the mapped documents
    reduce_documents_chain = ReduceDocumentsChain(
        # This is final chain that is called.
        combine_documents_chain=combine_documents_chain,
        # If documents exceed context for `StuffDocumentsChain`
        collapse_documents_chain=combine_documents_chain,
        # The maximum number of tokens to group documents into.
        token_max=4000,
    )

    # Combining documents by mapping a chain over them, then combining results
    map_reduce_chain = MapReduceDocumentsChain(
        # Map chain
        llm_chain=map_chain,
        # Reduce chain
        reduce_documents_chain=reduce_documents_chain,
        # The variable name in the llm_chain to put the documents in
        document_variable_name="docs",
        # Return the results of the map steps in the output
        return_intermediate_steps=False,
    )

    #Document Transformers in LangChain are crucial for manipulating documents,
    #performing tasks like splitting, combining, and filtering to adapt to model
    #context or specific application needs. The RecursiveCharacterTextSplitter
    #is a versatile tool allowing customization of parameters such as chunk size,
    #overlap, and starting index for efficient text segmentation.

    # Split documents into chunks using RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=4000, chunk_overlap=100
    )
    split_docs = text_splitter.split_documents(documents)

    # Run the chain
    result = map_reduce_chain.invoke(split_docs, return_only_outputs=True)
    return result['output_text']

In [ ]:
map_reduce_summarize_document()

PDF Token Count: 7228


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2227 > 1024). Running this sequence through the model will result in indexing errors


"1. The article discusses the impact of the Indian Premier League (IPL) on Test cricket in India.\n2. The authors argue that there is a gap in existing research on this topic and that this paper aims to fill that gap.\n3. The significance of the study includes assessing the current scenario of the popularity of Test cricket in India, understanding which format of the game is popular among youth, and examining whether fundamental cricket in India is influenced by IPL in terms of techniques and skills learning.\n4. The study will also analyze the current trend among people and take necessary steps according to that, help coaches in understanding the requirements of the game and developing measures and training plans accordingly, and provide information about the interest of youth cricketers.\n5. Objectives of the research: The main objective of the research is to determine how and what changes IPL has brought in Test cricket in India, how people perceive cricket in the country, and to as